Milestone 1

* Nama  : Raina Imtiyaz
* Batch : CODA-RMT-0012

Program ini dibuat untuk mengambil data produk dengan merk 'COSRX' dari e-commerce Sociolla dengan web scraping dan mengolah datanya menggunakan Pandas untuk analisis lebih lanjut.

# Import Library

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd

# Web Scraping

In [ ]:
driver = webdriver.Chrome()

nama_produk = []
harga_produk = []
rating = []
jumlah_terjual = []

# Mengambil data seluruh produk COSRX (terdapat 7 halaman)
for i in range(1, 8):
    url = f'https://www.sociolla.com/558_cosrx?tab=products&sort=-best-seller&page={i}'

    driver.get(url)

    # Menunggu 'product__name' muncul di web maksimal 10 detik
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "product__name"))
    )

    # Untuk scroll page
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    boxes = soup.find_all('div', {'class': 'loaded-item'})

    for box in boxes:
        # Nama produk    
        try:
            el_nama = box.find('a', {'class' : 'product__name'})
            nama_produk.append(el_nama.get_text())
        except:
            nama_produk.append(None)
        
        # Harga produk
        try:
            # Cari harga diskon dulu
            el_harga = box.find('h2', {'class' : 'product__price product__price_c-danger'})
            if el_harga:
                div = el_harga.find('div')
                if div and div.contents:
                    harga_text = div.contents[0].strip()
                else:
                    harga_text = el_harga.get_text()
                harga_produk.append(harga_text)
            else:
                # Kalau harga diskon tidak ada, ambil harga normal
                el_harga = box.find('h2', {'class' : 'product__price'})
                harga_produk.append(el_harga.get_text() if el_harga else None)
        except:
            harga_produk.append(None)
        
        # Rating produk
        try:
            el_rating = box.find('span', {'class': 'product__stars'})
            rating.append(el_rating.get_text())
        except:
            rating.append(None)
        
        # Jumlah produk terjual
        try:
            el_terjual = box.find('span', {'class': 'product__reviews'})
            jumlah_terjual.append(el_terjual.get_text())
        except:
            jumlah_terjual.append(None)

driver.quit()

# Dataframe Produk COSRX Hasil Web Scraping

In [17]:
df = pd.DataFrame()
df['nama_produk'] = nama_produk
df['harga_produk'] = harga_produk
df['rating'] = rating
df['jumlah_produk_terjual'] = jumlah_terjual
df

,nama_produk,harga_produk,rating,jumlah_produk_terjual
0,AHA 7 Whitehead Power Liquid,Rp206.250,4.5,"(3,4k)"
1,Salicylic Acid Daily Gentle Cleanser,Rp65.550-Rp116.250,4.7,"(10,4k)"
2,Low pH Good Morning Gel Cleanser,Rp58.650-Rp116.250,4.7,"(28,1k)"
3,Hyaluronic Acid Hydra Power Essence,Rp194.350,4.6,(634)
4,Advanced Snail 96 Mucin Power Essence,Rp202.300,4.6,"(16,3k)"
...,...,...,...,...
107,Pure Fit Cica Powder 2.0,Rp215.000,4.8,(7)
108,Pure Fit Cica Serum,Rp299.250,4.6,(233)
109,Pure Fit Cica Toner,Rp240.000,4.6,(335)
110,AC Collection Acne Hero kit Mild 2.0,Rp349.000,4.6,(247)


In [4]:
df.to_csv('sociolla_cosrx_products.csv', index=False)

# Eksplorasi Data

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   nama_produk            112 non-null    object
 1   harga_produk           112 non-null    object
 2   rating                 70 non-null     object
 3   jumlah_produk_terjual  70 non-null     object
dtypes: object(4)
memory usage: 3.6+ KB


NOTES:
1. Terdapat 42 missing value pada kolom `rating` dan `jumlah_produk_terjual`, akan ditelusuri apakah penyebab missing value pada rating adalah karena tidak ada produk yang terjual.
2. Tipe data pada `harga_produk`, `rating`, dan `jumlah_produk_terjual` perlu diubah menjadi numerik (`harga_produk` dan `jumlah_produk_terjual`ke tipe integer; `Rating` ke tipe float).
3. Jika dilihat pada dataframe, `harga_produk` masih ada yang dalam bentuk range harga (yang menunjukkan bahwa produk tersedia dalam berbagai ukuran) sehingga perlu diubah dengan mengambil minimum price sebagai `harga_produk`. Alasan pemilihan minimum price sebagai harga produk adalah karena harga tersebut merepresentasikan harga paling terjangkau dari produk tersebut; Pada beberapa produk, minimum price bisa merupakan harga produk satuan sedangkan maximum price adalah harga bundle; Jika ingin mengambil seluruh harga dalam range perlu membuka link per-produk dan per-ukuran.

## Missing Value pada kolom Rating dan Jumlah Produk Terjual

In [19]:
df_missing_rating = df[df['rating'].isna()]
df_missing_rating[['nama_produk', 'jumlah_produk_terjual']]

,nama_produk,jumlah_produk_terjual
23,Acne Pimple + Clear Fit Patch,None
31,Hydrium Green Tea Soothing Cream + Centella S...,None
32,Snail Series (Advanced Snail Peptide Eye Crea...,None
33,Advanced Snail Peptide Eye Cream + Advanced S...,None
34,AC Must Have (AC Collection Calming Foam Clea...,None
35,AC Mild Treatment Set (Ultimate Spot Cream + ...,None
36,Acne Pimple Master Patch (Set of 3),None
37,Clear Fit Master Patch (Set of 3),None
40,Oily Skin Bundle,None
41,Dry Skin Bundle,None


Bisa dinyatakan bahwa missing value pada kolom `rating` disebabkan oleh tidak adanya produk yang terjual, maka akan diisi nilai nol '0' saja.

In [ ]:
# Mengisi missing value pada kolom 'rating' dan 'jumlah_produk_terjual' dengan 0
df['rating'] = df['rating'].fillna(0)
df['jumlah_produk_terjual'] = df['jumlah_produk_terjual'].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   nama_produk            112 non-null    object
 1   harga_produk           112 non-null    object
 2   rating                 112 non-null    object
 3   jumlah_produk_terjual  112 non-null    object
dtypes: object(4)
memory usage: 3.6+ KB


Missing value pada kolom `rating` dan `jumlah_produk_terjual` sudah terisi dengan '0'

## Penanganan Harga Produk

Menghilangkan 'Rp' dan titik '.' pada harga, ambil harga minimum, dan ubah ke integer

In [ ]:
def remove_rp_dot(harga):
    '''
    Fungsi untuk menghilangkan karakter yang tidak diperlukan pada data harga, mengambil harga minimum sebagai harga, dan mengubah tipe data harga menjadi integer.

    Argumen:
    harga: string yang menunjukkan harga aau range harga suatu produk.

    Alur:
    1. Fungsi menerima input harga untuk dibersihkan.
    2. Fungsi menghapus 'Rp', titik, dan spasi di depan dan akhir harga.
    3. Jika data harga berupa range (terdapat atrip '-') maka data harga dipisah dengan strip sebagai pemisahnya, lalu ambil index ke-0 untuk mendapat harga minimum.
    4. Fungsi menghasilkan output integer dari harga yang sudah dibersihkan.

    Contoh Penggunaan:
    remove_rp_dot(Rp206.250)
    -> Output: 206250

    remove_rp_dot(Rp65.550-Rp116.250)
    -> Output: 65550
    '''
    harga = harga.replace("Rp", "").replace(".", "").strip()
    if "-" in harga:
        # Ambil harga paling rendah saja
        low = harga.split("-")[0]
        return int(low)
    return int(harga)

In [22]:
df['harga_produk'] = df['harga_produk'].apply(remove_rp_dot)

In [23]:
df[['nama_produk', 'harga_produk']]

,nama_produk,harga_produk
0,AHA 7 Whitehead Power Liquid,206250
1,Salicylic Acid Daily Gentle Cleanser,65550
2,Low pH Good Morning Gel Cleanser,58650
3,Hyaluronic Acid Hydra Power Essence,194350
4,Advanced Snail 96 Mucin Power Essence,202300
...,...,...
107,Pure Fit Cica Powder 2.0,215000
108,Pure Fit Cica Serum,299250
109,Pure Fit Cica Toner,240000
110,AC Collection Acne Hero kit Mild 2.0,349000


## Penanganan Rating

Ubah tipe data menjadi Float

In [24]:
df["rating"] = df["rating"].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   nama_produk            112 non-null    object 
 1   harga_produk           112 non-null    int64  
 2   rating                 112 non-null    float64
 3   jumlah_produk_terjual  112 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.6+ KB


## Penanganan Jumlah Produk Terjual

In [ ]:
def clean_jpt(qty):
    '''
    Fungsi untuk mengubah tipe data dan format penulisan data jumlah produk terjual.

    Argumen:
    qty = data dengan tipe string yang menunjukkan jumlah produk yang terjual.

    Alur:
    1. Fungsi menerima input berupa string jumlah produk terjual.
    2. Fungsi menghilangkan simbol '(' dan ')' lalu disimpan ke dalam variabel qty.
    3. Jika terdapat huruf 'k' dan koma ',' dalam data qty, fungsi menghilangkan huruf 'k' dan mengganti koma menjadi titik, mengubah datanya menjadi float, mengalikannya dengan 1000 (karena 'k' berarti ribu), lalu mengubah tipe datanya lagi menjadi integer.
    4. Jika data tidak kosong maka fungsi menghasilkan bilangan real dari qty, jika data kosong maka fungsi menghasilkan output None untuk produk tersebut.

    Contoh penggunaan:
    clean_jpt('15,3k')
    -> output: 15300
    '''
    qty = str(qty)
    
    # Hilangkan kurung ()
    qty = qty.replace('(', '').replace(')', '').strip()
    
    # Tangani ribuan 'k'
    if 'k' in qty:
        return int(float(qty.replace('k', '').replace(',', '.').strip()) * 1000)
    
    # Jika cuma angka
    try:
        return int(qty)
    except:
        return None


In [26]:
df['jumlah_produk_terjual'] = df['jumlah_produk_terjual'].apply(clean_jpt)

In [27]:
df.head()

,nama_produk,harga_produk,rating,jumlah_produk_terjual
0,AHA 7 Whitehead Power Liquid,206250,4.5,3400
1,Salicylic Acid Daily Gentle Cleanser,65550,4.7,10400
2,Low pH Good Morning Gel Cleanser,58650,4.7,28100
3,Hyaluronic Acid Hydra Power Essence,194350,4.6,634
4,Advanced Snail 96 Mucin Power Essence,202300,4.6,16300


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   nama_produk            112 non-null    object 
 1   harga_produk           112 non-null    int64  
 2   rating                 112 non-null    float64
 3   jumlah_produk_terjual  112 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 3.6+ KB


# Save Dataframe Bersih ke Dalam File CSV

In [16]:
df.to_csv('clean_sociolla_cosrx_products.csv', index=False)